## Define the big ABIDE sample
No PSM, no other limitations. Anything that passes QC.

In [1]:
import numpy as np
import pandas as pd
import pathlib as pal

In [4]:
#root_p = pal.Path('/mnt/stark_cisl/paper_16p22q/pheno/')
root_p = pal.Path('/home/surchs/data/paper_16p22q/pheno/')
sample_p = root_p / 'abide_1_complete.csv'
new_sample_p = root_p / 'abide_men.csv'

In [5]:
sample = pd.read_csv(sample_p)
# Strip whitespace from the ratings
sample['func'] = sample['func'].str.strip()
sample['anat'] = sample['anat'].str.strip()

In [6]:
sample.head()

,Civet_ID,Manual_Rating,Site,Subject,Session,Run,SUB_ID,Ratings,id_subject,status,...,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,BV,ct_available,fc_available,in_psm,Gotham_Severity
0,Caltech+0051456+session_1+anat_1,-1 (gradient too high around sensorimotor str...,Caltech,51456,session_1,anat_1,51456,2,X0051456,Fail,...,NaN,NaN,Closed,NaN,NaN,1218639.0,True,True,False,5.0
1,Caltech+0051457+session_1+anat_1,-1 (gradient too high around sm strip and to ...,Caltech,51457,session_1,anat_1,51457,2,X0051457,Fail,...,NaN,NaN,Closed,NaN,NaN,1430570.0,True,True,False,4.0
2,Caltech+0051458+session_1+anat_1,-1 (slightly too high gradient on sm strip,Caltech,51458,session_1,anat_1,51458,2,X0051458,Fail,...,NaN,NaN,Closed,NaN,NaN,1222213.0,True,True,False,10.0
3,Caltech+0051459+session_1+anat_1,-1 (more problematic in V1 than in sm strip; ...,Caltech,51459,session_1,anat_1,51459,2,X0051459,Maybe,...,NaN,NaN,Closed,NaN,NaN,1264665.0,True,True,False,7.0
4,Caltech+0051460+session_1+anat_1,-1 (V1 worse than sm strip),Caltech,51460,session_1,anat_1,51460,2,X0051460,Fail,...,NaN,NaN,Closed,NaN,NaN,1158337.0,True,True,False,7.0


In [7]:
sample.shape

(1107, 96)

In [8]:
qc_pass = (sample['func'].values=='OK') * (sample['anat'].values=='OK')
sample_qc = sample.loc[qc_pass]

In [9]:
men = sample_qc.loc[[True if row.SEX==1 else False for rid, row in sample_qc.iterrows()]]
# Get rid of everyone that isn't unique
duplicate_subjects = [sub for sub in men.Subject.unique() 
                      if men.loc[men.Subject==sub].shape[0]>1]
duplicate_mask = [False if row.Subject in duplicate_subjects else True for rid, row in men.iterrows()]
men_unique = men.loc[duplicate_mask]
good_sites = [site for site in men_unique.SITE_ID.unique() if men_unique.SITE_ID.value_counts()[site] > 10]
pheno_men = men_unique.loc[[True if row.SITE_ID in good_sites else False for rid, row in men_unique.iterrows()]]

In [10]:
#pheno_men.to_csv(str(new_sample_p), index=False)

In [11]:
pheno_men.shape

(459, 96)

In [12]:
pheno_men.groupby('SITE_ID').DX_GROUP.value_counts()

SITE_ID   DX_GROUP
KKI       Control     22
          Autism      14
LEUVEN_1  Autism       6
          Control      5
MAX_MUN   Control     11
          Autism       9
NYU       Control     66
          Autism      59
OLIN      Autism       9
          Control      8
SDSU      Control     13
          Autism       8
TRINITY   Control     25
          Autism      24
UCLA_1    Autism      28
          Control     24
UCLA_2    Autism       6
          Control      5
UM_1      Autism      10
          Control     10
USM       Autism      34
          Control     30
YALE      Autism      18
          Control     15
Name: DX_GROUP, dtype: int64

In [14]:
pheno_men.DX_GROUP.value_counts()

Control    234
Autism     225
Name: DX_GROUP, dtype: int64